# Initial Scraping Testing

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import bokeh
import matplotlib.pyplot as plt
import os
import lxml.html as lh

In [4]:
results_per_page = 10000
current_page = 1

url = 'http://www.vgchartz.com/games/games.php?page=' + str(current_page) +\
        '&results=' + str(results_per_page) +\
'&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership\
=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction\
=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&\
showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&\
showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'

Here's a cool scraper function I found, that I've modified to do what I need to do from VGChartz

In [23]:
def find_console_tags(soup):
    # Console tags are stored as images, so we find the image tag and record its 'alt' value as text
    consoles = list()
    for img in soup.find_all('img'):
        if 'images/consoles'in img['src']:
            # Cut file path elements from string
            console_tag = (img['src'][17:-6])
            consoles.append(img['alt'])
    return consoles


# Find the names of games from the links
def find_names_column(table_path):
    names_list = list()
    for row in table_path.xpath('.//tr'):
        for td in row.xpath('.//td'):
            if not td.find('a') is None:
                names_list.append(td.find('a').text.strip()) 
    return names_list

# Write a function that takes in a VGChartz URL and gives us all the data in their video game database
def scrape_vgchartz_videogame_db_page(url):
    try:
        response = requests.get(url)

        ### Check the Status
        assert(response.status_code == 200)," Website not OK " # status code = 200 => OK

        #Store the contents of the website under doc
        page=response.text
        soup = BeautifulSoup(page, "lxml")
        doc = lh.fromstring(response.content)

        # Selects the table with all the data in it on HTML using xpath
        target_table_path = doc.xpath('//*[@id="generalBody"]/table')[0]

        # Find column values that won't be scraped correctly with .text option
        names_list = find_names_column(target_table_path)
        consoles = find_console_tags(soup)

        # Parse non-image and non-URL info from the data table to a pandas DataFrame
        row_dict={}
        df=pd.DataFrame()
        row_list= list()
        for counter,row in enumerate(target_table_path.xpath(".//tr")):
            if counter > 2: # To skip header rows
                row_list=[td.text for td in row.xpath(".//td")]
                row_dict[counter] = row_list

        df=pd.DataFrame.from_dict(row_dict).transpose()
        df.columns = ['position','game','blank','console','publisher','developer','vgchart_score',\
                     'critic_score','user_score','total_shipped','total_sales',\
                      'na_sales','pal_sales','japan_sales','other_sales',\
                      'release_date','last_update']

        # Correct the console and game columns using scraped values

        df=df.reset_index().drop(columns = ['index','blank'])
        df['console'] = consoles
        df['game'] = names_list
        return df
    
    except:
        return pd.DataFrame()
    # We can 'hack' the URL to display any number of results per page. I'll leave it as an argument.
def scrape_all_vg_chartz_videogame_db(results_per_page):
    df = pd.DataFrame()
    current_page = 1
    games_left = True
    while games_left:
        url = 'http://www.vgchartz.com/games/games.php?page=' + str(current_page) +\
        '&results=' + str(results_per_page) + '&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership\
        =Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction\
        =DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&\
        showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&\
        showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'
        new_df = scrape_vgchartz_videogame_db_page(url)
        if not new_df.empty:
            df = df.append(new_df)
            print(f'page {current_page} done')
            current_page += 1
        else:
            break
    return df
# Run the code to scrape! I did 10,000 rows per page to speed things up.
df=scrape_all_vg_chartz_videogame_db(5000)

# Compress and store for later!
df.to_csv('vgsales.csv', sep=",", encoding='utf-8', index=False)
print("Wrote scraper data")

page 1 done
page 2 done
page 3 done
page 4 done
page 5 done
page 6 done
page 7 done
page 8 done
page 9 done
page 10 done
page 11 done
page 12 done
Wrote scraper data


**It worked** We have a bunch of data now, let's get started

In [68]:
df.sample(5)

,position,game,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
2001,47002,Strike Force Hydra,PS,Ignition Entertainment,Ignition Entertainment,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,23rd May 03,NaN
33,25034,Bird_Hunt,And,Evil Indie Games,Unknown,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,19th Mar 14,NaN
4384,19385,Cross Edge Dash,X360,Compile Heart,Compile Heart,N/A,N/A,N/A,NaN,0.01m,NaN,NaN,0.01m,NaN,01st Oct 09,NaN
2061,37062,Madden NFL 97,PC,EA Sports,High Score Productions,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,31st Aug 96,NaN
2020,57021,Summer Pockets,NS,Prototype,Key,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,20th Jun 19,26th Mar 19


Let's clean up a little

In [32]:
df = df.replace('N/A', np.nan)

In [69]:
df.sample(5)

,position,game,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
2321,47322,Super Mario Bros.: The Lost Levels,3DS,Nintendo,Unknown,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,27th Dec 12,NaN
450,45451,Ski-doo Snow X Racing,PS2,Valcon Games,ColdWood Interactive,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,02nd Aug 07,NaN
1819,26820,Chocobo Collection - Happy 10th Anniversary!,PS,Square,SquareSoft,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,22nd Dec 99,NaN
4565,4566,Tak and the Power of Juju,GBA,THQ,Avalanche Software,N/A,N/A,N/A,NaN,0.55m,0.39m,0.15m,NaN,0.01m,15th Oct 03,NaN
2754,57755,Valkyrie of Phantasm,PS4,Area Zero,Area Zero,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN,NaN,01st Dec 20,09th Jun 19


In [51]:
df.critic_score.value_counts()

N/A      51548
8.0        346
7.0        327
7.5        269
8.3        203
         ...  
1.7          2
1.4          1
1.8          1
1.3          1
1.2          1
Name: critic_score, Length: 90, dtype: int64

In [61]:
start_here = df[(df.console != 'Series') & (df.critic_score.apply(str.strip) != 'N/A')]

### Saving our progesss

Export to csv our scraping data

In [62]:
start_here.to_csv('../data/critic_scores.csv')

In [4]:
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")

In [67]:
start_here[start_here.game == 'Hardwood Spades']

,position,game,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
3485,33486,Hardwood Spades,XBL,Silver Creek Entertainment,Silver Creek Entertainment,N/A,6.2,N/A,NaN,NaN,NaN,NaN,NaN,NaN,08th Dec 05,NaN


## More random doodles


just some messing around scraper stuff

In [5]:
table = soup.find(id='generalBody')

In [20]:
infos = table.find_all('tr')[3:-1]

In [8]:
len(infos)

1001

In [8]:
columns = ['game', 'console', 'publisher','developer', 'vgchartz_score',
           'critic_score', 'user_score', 'total_shipped', 'release_date']

vgchartz = pd.DataFrame(columns=columns)

In [36]:
infos[105].find_all('td')[15]

<td align="center" width="75">07th Nov 06  </td>

In [47]:
infos[100].find_all('img')[1]['alt']

'X360'

In [23]:
lengths = [len(game) for game in infos]


In [51]:
for game in infos:
    stuff = game.find_all('td')
    index = stuff[0].text.strip()
    name = stuff[2].text.strip()
    console = game.find_all('img')[1]['alt'].strip()
    publisher = stuff[4].text.strip()
    developer = stuff[5].text.strip()
    shipped = stuff[9].text.strip()
    release = stuff[15].text.strip()
    vgchartz.loc[index] = [name, console, publisher, developer, 0, 0, 0, shipped, release]

In [85]:
no_series = vgchartz.loc[vgchartz.console != 'Series']

In [81]:
no_series.release_date = pd.to_datetime(no_series.release_date, dayfirst=True, errors='coerce')

In [94]:
def clean_sales(n):
    n = n.replace('m', '0000').replace('.', '')
    
    if n == 'N/A':
        return np.NaN
    
    return int(n)    

In [95]:
no_series.total_shipped = no_series.total_shipped.apply(clean_sales)

C:\Users\jaker\anaconda3\envs\learn-env\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [96]:
no_series

,game,console,publisher,developer,vgchartz_score,critic_score,user_score,total_shipped,release_date
20,Wii Sports,Wii,Nintendo,Nintendo EAD,0,0,0,82900000.0,19th Nov 06
57,Super Mario Bros.,NES,Nintendo,Nintendo EAD,0,0,0,40240000.0,18th Oct 85
58,Counter-Strike: Global Offensive,PC,Valve,Valve Corporation,0,0,0,40000000.0,21st Aug 12
62,Mario Kart Wii Read the review,Wii,Nintendo,Nintendo EAD,0,0,0,37320000.0,27th Apr 08
64,PLAYERUNKNOWN'S BATTLEGROUNDS,PC,PUBG Corporation,PUBG Corporation,0,0,0,36600000.0,21st Dec 17
...,...,...,...,...,...,...,...,...,...
996,Pokémon Stadium 2,N64,Nintendo,HAL Laboratory,0,0,0,2540000.0,28th Mar 01
997,Madden NFL 09,X360,EA Sports,EA Tiburon,0,0,0,NaN,12th Aug 08
998,Space Invaders,2600,Atari,Atari,0,0,0,NaN,01st Jan 78
999,Crazy Taxi,PS2,Acclaim Entertainment,Hitmaker,0,0,0,NaN,14th May 01
